<a href="https://colab.research.google.com/github/nobertomaciel/AI-UNIFACS/blob/main/cnn_mnist_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import tensorflow as tf
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

print(tf.__version__)

2.18.0


In [100]:
#importa os dados do dataset MNIST
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# conta a quantidade de dígitos (labels) do dataset (rótulos)
sns.countplot(y_train)

<Axes: ylabel='count'>

In [ ]:
# checar se existem valores Not a Number (NaN)
np.isnan(x_train).any()
np.isnan(x_test).any()
# caso existam dados NaN, devemos proceder com algum tipo de pré-processamento

In [ ]:
# realiza o ajuste do shape da CNN e normaliza os dados
input_shape = (28, 28, 1)

x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train=x_train / 255.0
x_train.shape

x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test=x_test/255.0
x_test.shape

In [ ]:
# troca os labels de categóricos para numéricos
y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [ ]:
# plota os dados para visualização
plt.imshow(x_train[100][:,:,0])
print(y_train[100])

In [ ]:
# define os parâmetros iniciais da rede: o tamanho das camadas de convolução internas, o número de classes e épocas
batch_size = 64
num_classes = 10
epochs = 5

In [ ]:
# inicialliza a rede convolucional

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
# realiza o treinamento da CNN
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.995):
      print("\nReached 99.5% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.1,
                    callbacks=[callbacks])

In [ ]:
# plota os valores de performance da rede
fig, ax = plt.subplots(2,1)
print("VALORES DE ERRO E ACURÁCIA -------------------------------------------------------------------")
print("TREINAMENTO", history.history['loss'])
print("VALIDAÇÃO", history.history['val_loss'])
print("TREINAMENTO", history.history['acc'])
print("VALIDAÇÃO", history.history['val_acc'])
print("---------------------------------------------------------------------------------------------")

ax[0].plot(history.history['loss'], color='b', label="Training Loss")
ax[0].plot(history.history['val_loss'], color='r', label="Validation Loss")
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['acc'], color='b', label="Training Accuracy")
ax[1].plot(history.history['val_acc'], color='r',label="Validation Accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)

In [ ]:
# Predict the values from the testing dataset
Y_pred = model.predict(x_test)
# Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred,axis = 1)
# Convert testing observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1)
# compute the confusion matrix
confusion_mtx = tf.math.confusion_matrix(Y_true, Y_pred_classes)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, annot=True, fmt='g')

In [ ]:
# o dataset de treinamento possui 60000 imagens 28x28 pixels
print("Amostra dos dados de treinamento:")
print("-----------------------------------------")
print(x_train.shape)
print(x_train[59999][27][27])
print(x_train[59999][27][27][0])

# o dataset de teste possui 60000 imagens 28x28 pixels
print("\nAmostra dos dados de teste:")
print("-----------------------------------------")
print(x_test.shape)
print(x_test[9999][27][27][0])
print(x_test[0][10][11])
print(x_test[0][10][27])
print(x_test[0][27][27])
print(x_test[9999][27][27][0])
print(x_test[0][10])

In [ ]:
Y_pred = model.predict(x_test[0:9999])
Y_pred_classes = np.argmax(Y_pred,axis = 1)

In [ ]:
print(Y_pred.shape)
Y_pred

In [ ]:
print(Y_pred_classes.shape)
Y_pred_classes